In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt


gdims = 2
N = 50
brain_radius = 0.1
fluid_radius = brain_radius*1.2
ventricle_radius = 0.03
aqueduct_width = 0.005
canal_width = 0.02
canal_length = 0.2

# subdomain ids
fluid_id = 2
porous_id = 1

# boundary ids
interface_id = 1
rigid_skull_id = 2
spinal_outlet_id = 3

if gdims==2:

    brain = Circle(Point(0,0), brain_radius)
    ventricle = Circle(Point(0,0), ventricle_radius)
    aqueduct = Rectangle(Point(-aqueduct_width/2, -brain_radius), Point(aqueduct_width/2, 0))
    brain = brain - ventricle - aqueduct

    spinal_canal = Rectangle(Point(-canal_width/2, -canal_length), Point(canal_width/2, 0))
    fluid = Circle(Point(0,0), fluid_radius) + spinal_canal
    domain = fluid + brain
    domain.set_subdomain(fluid_id, fluid)
    domain.set_subdomain(porous_id, brain)
    
elif gdims==3:
    brain = Sphere(Point(0,0,0), brain_radius)
    ventricle = Sphere(Point(0,0,0), ventricle_radius)
    
    aqueduct = Cylinder(Point(0,0,0), Point(0, -brain_radius),aqueduct_width, aqueduct_width )
    brain = brain - ventricle - aqueduct
    spinal_canal = Cylinder(Point(0,0,0), Point(0, -canal_length),canal_width, canal_width )
    fluid = Sphere(Point(0,0,0), fluid_radius) + spinal_canal
    domain = fluid + brain
        

mesh = generate_mesh(domain, N)
mf = MeshFunction("size_t", mesh, gdims, mesh.domains())

if gdims==3:
    mf.set_all(fluid_id)
    inner_brain_sphere = CompiledSubDomain("x[0]*x[0] + x[1]*x[1] + x[2]*x[2] < br*br", br=brain_radius)
    inner_ventricle_sphere = CompiledSubDomain("x[0]*x[0] + x[1]*x[1] + x[2]*x[2] < vr*vr", vr=ventricle_radius)
    aquaeduct_dom = CompiledSubDomain("x[0]*x[0] + x[1]*x[1] < ar*ar", ar=aqueduct_width*3)
    
    inner_brain_sphere.mark(mf, porous_id)
    inner_ventricle_sphere.mark(mf, fluid_id)
    aquaeduct_dom.mark(mf, fluid_id)
    
#plt.figure(figsize=(9,9))
#dolfin.plot(mf, "Subdomains")

In [ ]:
boundaries = MeshFunction("size_t", mesh, gdims - 1, 0)

# set internal interface
for f in facets(mesh):
    domains = []
    for c in cells(f):
        domains.append(mf[c])

    domains = list(set(domains))
    if len(domains) > 1:
        boundaries[f] = interface_id
        
# set rigid skull boundary

rigid_skull = CompiledSubDomain("on_boundary",
                                canal_length=canal_length)

rigid_skull.mark(boundaries, rigid_skull_id)

# set spinal outlet 
spinal_outlet = CompiledSubDomain("near(x[1], - canal_length)",
                                canal_length=canal_length)
spinal_outlet.mark(boundaries, spinal_outlet_id)

boundaries_outfile = XDMFFile(f"../meshes/ideal_brain_boundaries{gdims}D_N{N}.xdmf")
subdomains_outfile = XDMFFile(f"../meshes/ideal_brain_subdomains{gdims}D_N{N}.xdmf")

boundaries_outfile.write(boundaries)
subdomains_outfile.write(mf)
boundaries_outfile.close()
subdomains_outfile.close()